In [ ]:
import os
os.makedirs('utils', exist_ok=True)

sentiment_code = '''
import requests
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import praw
from bs4 import BeautifulSoup

def get_yahoo_finance_headlines(ticker):
    """Get headlines from Yahoo Finance"""
    try:
        url = f"https://finance.yahoo.com/quote/{ticker}/news"
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
        response = requests.get(url, headers=headers, timeout=10)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            headlines = []

            news_items = soup.find_all('h3', class_='Mb(5px)')
            for item in news_items[:10]:
                headline = item.get_text().strip()
                if headline:
                    headlines.append(headline)

            return headlines
        return []
    except Exception as e:
        print(f"Error fetching Yahoo headlines: {e}")
        return []

def get_reddit_headlines(ticker, client_id, client_secret):
    """Get headlines from Reddit"""
    try:
        reddit = praw.Reddit(
            client_id=client_id,
            client_secret=client_secret,
            user_agent="MarketPulse/1.0"
        )

        headlines = []
        subreddits = ['stocks', 'investing', 'SecurityAnalysis', 'ValueInvesting']

        for sub_name in subreddits:
            try:
                subreddit = reddit.subreddit(sub_name)
                for post in subreddit.search(ticker, limit=5):
                    headlines.append(post.title)
            except:
                continue

        return headlines
    except Exception as e:
        print(f"Error fetching Reddit headlines: {e}")
        return []

def analyze_sentiment(headlines):
    """Analyze sentiment of headlines"""
    analyzer = SentimentIntensityAnalyzer()
    results = []

    for headline in headlines:
        score = analyzer.polarity_scores(headline)
        compound = score['compound']

        if compound >= 0.05:
            label = "Positive"
        elif compound <= -0.05:
            label = "Negative"
        else:
            label = "Neutral"

        results.append({
            'headline': headline,
            'score': compound,
            'label': label
        })

    return results

def summarize_sentiment(results):
    """Summarize overall sentiment"""
    if not results:
        return 0, "Neutral"

    avg_score = sum(r['score'] for r in results) / len(results)

    if avg_score >= 0.05:
        label = "Positive"
    elif avg_score <= -0.05:
        label = "Negative"
    else:
        label = "Neutral"

    return avg_score, label
'''

with open('utils/sentiment_analysis.py', 'w') as f:
    f.write(sentiment_code)

print("Created utils/sentiment_analysis.py")

In [ ]:
report_code = '''
from fpdf import FPDF
import datetime

class PDF(FPDF):
    def __init__(self):
        super().__init__()
        self.set_auto_page_break(auto=True, margin=15)

    def header(self):
        self.set_font('Arial', 'B', 15)
        self.cell(0, 10, 'MarketPulse - Portfolio Analysis Report', 0, 1, 'C')
        self.ln(10)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Page {self.page_no()} - Generated on {datetime.date.today()}', 0, 0, 'C')

    def add_stock_section(self, ticker, sentiment_label, sentiment_score):
        self.ln(5)
        self.set_font('Arial', 'B', 14)
        self.cell(0, 10, f'Stock Analysis: {ticker}', 0, 1)

        self.set_font('Arial', '', 12)
        self.cell(0, 8, f'Sentiment: {sentiment_label}', 0, 1)
        self.cell(0, 8, f'Sentiment Score: {sentiment_score:.3f}', 0, 1)
        self.ln(5)

    def add_text(self, text,